<a href="https://colab.research.google.com/github/brandon0824/HungyiML/blob/master/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!nvidia-smi

In [12]:
import numpy as np

np.random.seed(0)   # 保证生成相同的随机数
X_train_fpath = './drive/My Drive/Hung-yi Lee DLML HW/hw2/data/X_train'
Y_train_fpath = './drive/My Drive/Hung-yi Lee DLML HW/hw2/data/Y_train'
X_test_fpath = './drive/My Drive/Hung-yi Lee DLML HW/hw2/data/X_test'
output_fpath = './drive/My Drive/Hung-yi Lee DLML HW/hw2/output_{}.csv'

# Parse csv files to numpy array
with open(X_train_fpath) as f:
    next(f) # 迭代器 返回下一个f
    X_train = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = float)  # 第一列id不要 去除
with open(Y_train_fpath) as f:
    next(f)
    Y_train = np.array([line.strip('\n').split(',')[1] for line in f], dtype = float)
with open(X_test_fpath) as f:
    next(f)
    X_test = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = float)

def _normalize(X, train = True, specified_column = None, X_mean = None, X_std = None):
    # This function normalizes specific columns of X.
    # The mean and standard variance of training data will be reused when processing testing data.
    #
    # Arguments:
    #     X: data to be processed
    #     train: 'True' when processing training data, 'False' for testing data
    #     specific_column: indexes of the columns that will be normalized. If 'None', all columns
    #         will be normalized.
    #     X_mean: mean value of training data, used when train = 'False'
    #     X_std: standard deviation of training data, used when train = 'False'
    # Outputs:
    #     X: normalized data
    #     X_mean: computed mean value of training data
    #     X_std: computed standard deviation of training data

    if specified_column == None:    # 所有列都会被正则化
        specified_column = np.arange(X.shape[1]) # X.shape[1]为X的列数 生成一个[0,1,2...X列长度-1]的np数组
    if train:
        X_mean = np.mean(X[:, specified_column] ,0).reshape(1, -1) # 求每列的均值
        X_std  = np.std(X[:, specified_column], 0).reshape(1, -1) # 求每列的方差

    X[:,specified_column] = (X[:, specified_column] - X_mean) / (X_std + 1e-8) # 防止分母为0
    # 取所有行的 specified_column 列 参考：https://zhuanlan.zhihu.com/p/144609911 预处理过程
    return X, X_mean, X_std

def _train_dev_split(X, Y, dev_ratio = 0.25):
    # This function spilts data into training set and development set.
    train_size = int(len(X) * (1 - dev_ratio))
    return X[:train_size], Y[:train_size], X[train_size:], Y[train_size:]

# Normalize training and testing data
X_train, X_mean, X_std = _normalize(X_train, train = True)
X_test, _, _= _normalize(X_test, train = False, specified_column = None, X_mean = X_mean, X_std = X_std)
    
# Split data into training set and development set
dev_ratio = 0.1
X_train, Y_train, X_dev, Y_dev = _train_dev_split(X_train, Y_train, dev_ratio = dev_ratio)

train_size = X_train.shape[0]
dev_size = X_dev.shape[0]
test_size = X_test.shape[0]
data_dim = X_train.shape[1]
print('Size of training set: {}'.format(train_size))
# train.csv 54255个id 54255*(1-0.1)=48830
print('Size of development set: {}'.format(dev_size))
# train.csv 54255个id 54255*0.1=5426
print('Size of testing set: {}'.format(test_size))
# X_test 27622个id
print('Dimension of data: {}'.format(data_dim))
# X_train X_test的列数

Size of training set: 48830
Size of development set: 5426
Size of testing set: 27622
Dimension of data: 510
